# Implementacion de servicios de aprendizaje automatico en tiempo real con Azure ML

- **Inferencia**: Uso de un modelo entrenado para predecir etiquetas para datos nuevos (no usados durante el entrenamiento).

- **Servicio de Inferencia en Tiempo Real**: Permite a las aplicaciones solicitar predicciones inmediatas al modelo para datos individuales o pequeños conjuntos.

Despliegue del Modelo:

- **Contenedorizado en AKS (Azure Kubernetes Services)**: Plataforma de orquestación de contenedores para implementar y administrar aplicaciones en contenedores.

- **Servicio de Inferencia**: Punto de acceso para que las aplicaciones consuman el modelo.

Un modelo puede implementarse como un servicio web en tiempo real en varios destinos, incluyendo localmente, Azure ML, ACI, AKS, una función de Azure o un módulo IoT. Azure ML contenedores para la implementación, empaquetando el modelo y el código en una imagen que se puede implementar en un contenedor en el destino seleccionado.

Nota

    La implementación en un servicio local, una instancia informática o una ACI es una buena opción para las pruebas y el desarrollo. Para producción, se debe implementar en un destino que satisfaga las necesidades específicas de rendimiento, escalabilidad y seguridad de la arquitectura de la aplicación.

Para implementar un modelo como un servicio de inferencia en tiempo real, debe realizar las siguientes tareas:

#### 1. Registro de un modelo entrenado
Después de entrenar correctamente un modelo, debe registrarlo en el área de trabajo de Azure ML. Su servicio en tiempo real podrá cargar el modelo cuando sea necesario.

Para registrar un modelo a partir de un archivo local, puede utilizar el método `register` del  objeto Model como se muestra:

In [ ]:
# Cuando tienes un modelo que se ha entrenado fuera de Azure ML y quieres registrar el modelo en tu espacio de trabajo.

from azureml.core import Model

classification_model = Model.register(workspace=ws,                         # Registramos el modelo en el espacio de trabajo
                       model_name='classification_model',
                       model_path='model.pkl',                              # ruta local
                       description='A classification model')

Alternativamente, si tenemos referencia al `run` utilizado para entrenar el modelo, podemos utilizar su método `register_model`

In [ ]:
# run en Azure ML se refiere a una ejecución individual de un script de entrenamiento o un pipeline.
# Cuando se llama a run.register_model, el modelo se registra junto con metadatos sobre el Run específico, como los parámetros de entrenamiento utilizados. 
# Esto puede ser útil para rastrear cómo se creó el modelo.

run.register_model( model_name='classification_model',                      # Registramos el modelo en el espacio de trabajo
                    model_path='outputs/model.pkl',                         # run ruta de salida
                    description='A classification model')

#### 2. Definir una configuración de inferencia

Los modelos que se implementan como un servicio, constan de:

1. Un script para cargar el modelo y devolver predicciones para los datos enviados.
2. Un entorno en el que se ejecutará el script.

##### Crear un script de entrada

Creando la secuencia de comandos de entrada (tambien llamada **secuencia de comandos de puntuación**), para el servicio como un archivo de Python (.py), necesitamos de:

   - `init()`: Se llama cuando se inicializa el servicio.
   - `run(raw_data)`: Se llama cuando se envían nuevos datos al servicio.

Normalmente, se usa la función `init` para cargar el modelo desde el registro de modelos y se usa la función `run` para generar predicciones a partir de los datos de entrada.

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# llamamos a la función init() para cargar el modelo en la memoria cuando se inicia el contenedor
def init():
    global model
    model_path = Model.get_model_path('classification_model')                   # Obtenemos la ruta del modelo y lo cargamos en memoria
    model = joblib.load(model_path)


# llamamos a la función run() para obtener una predicción para los datos de entrada cuando se realiza una solicitud al servicio web
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])                               # Convertimos los datos de entrada en un array de numpy
    predictions = model.predict(data)                                           # Realizamos las predicciones
    return predictions.tolist()                                                 # Devolvemos las predicciones como una lista serializable en JSON

##### Crear un entorno

Requerimos de un entorno de Python en el que ejecutar el script de entrada, que se puede configurar mediante el archivo de configuración de Conda. 

Una manera sencilla de crear este archivo es usar una clase `CondaDependencies` para crear un entorno predeterminado (que incluye el paquete `azureml-defaults` y paquetes de uso común como `numpy` y `pandas`, pero podemos agregar cualquier otro paquete necesario), a continuación, serealizamos el entorno en una cadena y lo guardamos:

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Creamos un objeto CondaDependencies y agregamos las dependencias necesarias
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Guardamos el archivo de especificación de entorno
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

##### Combinar el script y el entorno en un InferenceConfig

Después de crear el script de entrada y el archivo de configuración del entorno, podemos combinarlos en un `InferenceConfig` para el servicio de la siguiente manera

In [ ]:
from azureml.core.model import InferenceConfig

# Creamos un objeto InferenceConfig con la configuración necesaria
classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

##### Definir una configuración de implementación

Con el script y el archivo de entorno, ahora debemos configurar el proceso en el que se implementará el servicio. 

Si vamos a realizar la implementación en un clúster de AKS, tenemos que crear el clúster y un destino de proceso para él antes de realizar la implementación

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute

# Creamos un objeto AksCompute con la configuración necesaria para el clúster de AKS
cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Una vez creado el destino del proceso, podemos definir la configuración de implementación. Esta configuración establece la especificación del proceso, específica para el destino, para la implementación en contenedores.

In [ ]:
from azureml.core.webservice import AksWebservice

# Creamos un objeto AksWebservice con la configuración necesaria para el servicio web
classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

Para configurar una implementación de ACI, el código es similar. 

No necesitas crear un destino de proceso de ACI explícitamente y debes usar la clase `deploy_configuration` del espacio de nombres `azureml.core.webservice.AciWebservice`. De manera similar, puedes usar el espacio de nombres `azureml.core.webservice.LocalWebservice` para configurar un servicio local basado en Docker.

Si queremos implementar un modelo en una función de Azure, no necesitas crear una configuración de implementación. En su lugar, debes empaquetar el modelo según el tipo de desencadenador de función que quieras usar. Esta funcionalidad está en versión preliminar en el momento de escribir este artículo. Para obtener más información, consulta la sección [“Implementación de un modelo de aprendizaje automático en Azure Functions”](https://learn.microsoft.com/es-es/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=azure-cli) en la documentación de Azure ML.

##### Implementar el modelo
Una vez preparada toda la configuración, podemos implementar el modelo. La forma más fácil de hacerlo es llamar al método deploy de la clase `Model`, de la siguiente manera:

In [2]:
from azureml.core.model import Model

model = ws.models['classification_model']                                       # Obtenemos el modelo por su nombre
service = Model.deploy(workspace=ws,                                            # Desplegamos el servicio web en el clúster de AKS
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)                                 # Esperamos a que se complete el despliegue

ModuleNotFoundError: No module named 'azureml'

En el caso de ACI o servicios locales, puede omitir el parámetro `deployment_target` (o establecerlo en **None**).

[Más into sobre la implementación de modelos con Azure Machine Learning](https://learn.microsoft.com/es-es/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=azure-cli)

#### 3. Consumo de un servicio de inferencia en tiempo real

Después de implementar el servicio en tiempo real, puede consumirlo desde aplicaciones cliente para predecir etiquetas para nuevos casos de datos.

##### Uso del SDK de Azure Machine Learning

Para realizar pruebas, puede usar el SDK de Azure ML para llamar a un servicio web a través del método run de un objeto `WebService` que haga referencia al servicio implementado. Normalmente, los datos se envían al método `run` en formato JSON con la siguiente estructura:

In [ ]:
{
  "data":[
      [0.1,2.3,4.1,2.0],  // 1st case
      [0.2,1.8,3.9,2.1],  // 2nd case
      ...
  ]
}

La respuesta del método `run` sera una colección JSON con una predicción para cada caso que se envió en los datos.

In [ ]:
# En este ejemplo se llama a un servicio y se muestra la respuesta

import json

# matriz de datos nuevos
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]


json_data = json.dumps({"data": x_new})                     # Convertimos la matriz en un formato JSON serializable

response = service.run(input_data = json_data)              # Llamamos al servicio web con los datos de entrada

predictions = json.loads(response)                          # Convertimos la respuesta en un objeto JSON

for i in range(len(x_new)):                                 # Mostramos las predicciones para cada conjunto de datos
    print (x_new[i], predictions[i])

##### Uso de un punto de conexión REST 

En producción, la mayoría de las aplicaciones cliente no incluirán el SDK de Azure ML y consumirán el servicio a través de su interfaz REST. 

Podemos determinar el punto de conexión de un servicio implementado en Azure ML Studio o recuperar la propiedad `scoring_uri` del objeto Webservice en el SDK

In [ ]:
# En este ejemplo se muestra cómo obtener la dirección URL del servicio web

endpoint = service.scoring_uri
print(endpoint)

Con el punto de conexión conocido, puede usar una solicitud HTTP POST con datos JSON para llamar al servicio. En el siguiente ejemplo se muestra cómo hacerlo con Python:

In [ ]:
import requests
import json

x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

json_data = json.dumps({"data": x_new})                         # Convertimos la matriz en un formato JSON serializable

request_headers = { 'Content-Type':'application/json' }         # Set the content type in the request headers

response = requests.post(url = endpoint,                        # Realizamos la solicitud POST al servicio web
                         data = json_data,
                         headers = request_headers)

predictions = json.loads(response.json())                       # Convertimos la respuesta en un objeto JSON

for i in range(len(x_new)):                                     # Imprimimos las predicciones para cada conjunto de datos
    print ((x_new[i]), predictions[i])

##### Autenticación

En producción, es probable que queramos restringir el acceso a los servicios mediante la aplicación de autenticación. Hay dos tipos de autenticación que se pueden aplicar:
- **Clave**: las solicitudes se autentican especificando la clave asociada al servicio.
- **Token**: se autentican proporcionando un token web JSON (JWT).

De forma predeterminada, la autenticación está deshabilitada para los servicios ACI y se establece en la autenticación basada en claves para los servicios de AKS (para los que las claves principal y secundaria se generan automáticamente). Opcionalmente, puede configurar un servicio de AKS para usar la autenticación basada en tokens (que no es compatible con los servicios ACI).

Suponiendo que tenemos una sesión autenticada establecida con el área de trabajo, podemos recuperar las claves del servicio mediante el método `get_keys` del objeto WebService asociado al servicio:

In [ ]:
primary_key, secondary_key = service.get_keys()

En el caso de la autenticación basada en tokens, la aplicación cliente debe usar la autenticación de entidad de servicio para comprobar su identidad a través de Azure Active Directory (Azure AD) y llamar al  método `get_token` del servicio para recuperar un token de tiempo limitado.

Para realizar una llamada autenticada al punto de conexión REST del servicio, debemos incluir la clave o el token en el encabezado de la solicitud de la siguiente manera

In [ ]:
import requests
import json

x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

json_data = json.dumps({"data": x_new})

request_headers = { "Content-Type":"application/json",                      # incluimos la clave de autenticación en la solicitud
                    "Authorization":"Bearer " + key_or_token }

response = requests.post(url = endpoint,                                    # llama al servicio web con los datos de entrada
                         data = json_data,
                         headers = request_headers)

predictions = json.loads(response.json())

for i in range(len(x_new)):
    print ((x_new[i]), predictions[i])

#### 5. Implementación de un modelo como servicio web en tiempo real

Destinos de proceso
- Proceso local
- Instancia de proceso de Azure Machine Learning
- Instancia de Azure Container Instance (ACI)
- Clúster de Azure Kubernetes Service (AKS)
- Función de Azure
- Módulo de Internet de las cosas (IoT)

Mecanismo de implementación
- Contenedores
- Empaquetado del modelo y el código como una imagen
- Implementación en un contenedor en el destino elegido

Consideraciones:
- Pruebas y desarrollo: servicio local, instancia informática o ACI
- Producción: destino que cumpla con los requisitos de rendimiento, escalabilidad y seguridad de la aplicación

Tareas para implementar un servicio de inferencia en tiempo real:
1. **Registrar el modelo y el entorno**: Almacenar el modelo y su entorno de ejecución en Azure Machine Learning.
2. **Crear una configuración de inferencia**: Especificar cómo se ejecuta el modelo en tiempo real.
3. **Crear una configuración de destino**: Seleccionar el destino de proceso y las opciones de implementación.
4. **Implementar el servicio**: Implementar el modelo como un servicio web en el destino elegido.
5. **Probar el servicio**: Enviar solicitudes de prueba al servicio y verificar las predicciones.